In [1]:
from pyspark.sql import SQLContext
from pyspark import SparkContext

sc = SparkContext()
sqlContext = SQLContext(sc)

21/08/28 19:30:08 WARN Utils: Your hostname, lukas-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
21/08/28 19:30:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/08/28 19:30:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# Read data from ``data/hotels_bahamas_min.csv``

Read minimal data about hotels in the Bahamas from the CSV file an store them in temporary table ``hotels``.

In [2]:
df = sqlContext.read.format('csv').options(header='true', inferschema='true').load('/home/lukas/spark/data/hotels_bahamas_min.csv')
df.registerTempTable("hotels")
sqlContext.sql("SELECT * FROM hotels").show()
sqlContext.sql("SELECT * FROM hotels SKYLINE OF price MIN, distance MIN").show()

+---+-----+--------+
| id|price|distance|
+---+-----+--------+
|  1|  257|     600|
|  2|  323|     350|
|  3|  276|       0|
|  4|  234|     100|
|  5|  265|     250|
+---+-----+--------+

+---+-----+--------+
| id|price|distance|
+---+-----+--------+
|  1|  257|     600|
|  2|  323|     350|
|  3|  276|       0|
|  4|  234|     100|
|  5|  265|     250|
+---+-----+--------+



In [3]:
sqlContext.sql("SELECT * FROM hotels").explain("extended")
sqlContext.sql("SELECT * FROM hotels SKYLINE OF price MIN, distance MIN").explain()
sqlContext.sql("SELECT * FROM hotels SKYLINE OF price MIN, distance MIN").explain("extended")

== Parsed Logical Plan ==
'Project [*]
+- 'UnresolvedRelation [hotels], [], false

== Analyzed Logical Plan ==
id: int, price: int, distance: int
Project [id#16, price#17, distance#18]
+- SubqueryAlias hotels
   +- Relation[id#16,price#17,distance#18] csv

== Optimized Logical Plan ==
Relation[id#16,price#17,distance#18] csv

== Physical Plan ==
FileScan csv [id#16,price#17,distance#18] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/home/lukas/spark/data/hotels_bahamas_min.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<id:int,price:int,distance:int>

== Physical Plan ==
*(1) (unknown) SkylineInput [3]: [id#16, price#17, distance#18]; Skyline dimensions: price#17 MIN, distance#18 MIN; Distinct: false
+- FileScan csv [id#16,price#17,distance#18] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/home/lukas/spark/data/hotels_bahamas_min.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<id:int,

In [4]:
df.filter("price > 200").skyline(["price", "distance"], minMaxDiff=["min", "min"]).show()
df.filter("price > 200").skyline(["price", "distance"], minMaxDiff=["min", "min"]).explain("extended")

+---+-----+--------+
| id|price|distance|
+---+-----+--------+
|  1|  257|     600|
|  2|  323|     350|
|  3|  276|       0|
|  4|  234|     100|
|  5|  265|     250|
+---+-----+--------+

== Parsed Logical Plan ==
'SkylineOperator SkylineIsNotDistinct, ['price MIN, 'distance MIN]
+- Filter (price#17 > 200)
   +- Relation[id#16,price#17,distance#18] csv

== Analyzed Logical Plan ==
id: int, price: int, distance: int
SkylineOperator SkylineIsNotDistinct, [price#17 MIN, distance#18 MIN]
+- Filter (price#17 > 200)
   +- Relation[id#16,price#17,distance#18] csv

== Optimized Logical Plan ==
SkylineOperator SkylineIsNotDistinct, [price#17 MIN, distance#18 MIN]
+- Filter (isnotnull(price#17) AND (price#17 > 200))
   +- Relation[id#16,price#17,distance#18] csv

== Physical Plan ==
*(1) (unknown) SkylineInput [3]: [id#16, price#17, distance#18]; Skyline dimensions: price#17 MIN, distance#18 MIN; Distinct: false
+- *(1) Filter (isnotnull(price#17) AND (price#17 > 200))
   +- FileScan csv [id#1

In [5]:
df.filter("price > 200").skylineDistinct(["price", "distance"], minMaxDiff=["min", "min"]).show()
df.filter("price > 200").skylineDistinct(["price", "distance"], minMaxDiff=["min", "min"]).explain("extended")

+---+-----+--------+
| id|price|distance|
+---+-----+--------+
|  1|  257|     600|
|  2|  323|     350|
|  3|  276|       0|
|  4|  234|     100|
|  5|  265|     250|
+---+-----+--------+

== Parsed Logical Plan ==
'SkylineOperator SkylineIsDistinct, ['price MIN, 'distance MIN]
+- Filter (price#17 > 200)
   +- Relation[id#16,price#17,distance#18] csv

== Analyzed Logical Plan ==
id: int, price: int, distance: int
SkylineOperator SkylineIsDistinct, [price#17 MIN, distance#18 MIN]
+- Filter (price#17 > 200)
   +- Relation[id#16,price#17,distance#18] csv

== Optimized Logical Plan ==
SkylineOperator SkylineIsDistinct, [price#17 MIN, distance#18 MIN]
+- Filter (isnotnull(price#17) AND (price#17 > 200))
   +- Relation[id#16,price#17,distance#18] csv

== Physical Plan ==
*(1) (unknown) SkylineInput [3]: [id#16, price#17, distance#18]; Skyline dimensions: price#17 MIN, distance#18 MIN; Distinct: true
+- *(1) Filter (isnotnull(price#17) AND (price#17 > 200))
   +- FileScan csv [id#16,price#17

In [6]:
df.filter("price > 200").sort(df.price.desc()).explain("extended")
df.filter("price > 200").skylineDistinct(df.price.smin(), df.distance.smax()).explain("extended")

type <class 'py4j.java_gateway.JavaObject'> of [JavaObject id=o74]
== Parsed Logical Plan ==
Sort [price#17 DESC NULLS LAST], true
+- Filter (price#17 > 200)
   +- Relation[id#16,price#17,distance#18] csv

== Analyzed Logical Plan ==
id: int, price: int, distance: int
Sort [price#17 DESC NULLS LAST], true
+- Filter (price#17 > 200)
   +- Relation[id#16,price#17,distance#18] csv

== Optimized Logical Plan ==
Sort [price#17 DESC NULLS LAST], true
+- Filter (isnotnull(price#17) AND (price#17 > 200))
   +- Relation[id#16,price#17,distance#18] csv

== Physical Plan ==
*(2) Sort [price#17 DESC NULLS LAST], true, 0
+- Exchange rangepartitioning(price#17 DESC NULLS LAST, 200), ENSURE_REQUIREMENTS, [id=#179]
   +- *(1) Filter (isnotnull(price#17) AND (price#17 > 200))
      +- FileScan csv [id#16,price#17,distance#18] Batched: false, DataFilters: [isnotnull(price#17), (price#17 > 200)], Format: CSV, Location: InMemoryFileIndex[file:/home/lukas/spark/data/hotels_bahamas_min.csv], PartitionFilter